# DOLFINx meshes

DOLFINx provides a range of simple built-in meshes that are useful for quick testing and solving basic problems. You can find more information about the available mesh types, as well as tools for creating, refining, and marking meshes, in the [DOLFINx mesh documentation](https://docs.fenicsproject.org/dolfinx/v0.9.0/python/generated/dolfinx.mesh.html).

In this tutorial, we will cover the following:

- How to create basic meshes using DOLFINx  
- How to apply mesh transformations  
- How to use these meshes in a FESTIM simulation  
- How to define and label subdomains on DOLFINx meshes  

## Built-in DOLFINx meshes

The `create_unit_square` function in DOLFINx generates a structured, 2D mesh of the unit square, divided into a specified number of cells in the x and y directions. This is useful for simple test cases and provides a convenient starting point for setting up simulations.

In [34]:
from dolfinx.mesh import create_unit_square
from mpi4py import MPI

nx, ny = 10, 10  # Number of divisions in x and y directions
mesh = create_unit_square(MPI.COMM_WORLD, nx, ny)

In [35]:
from dolfinx import plot
import pyvista

pyvista.start_xvfb()
pyvista.set_jupyter_backend("html")


tdim = mesh.topology.dim

mesh.topology.create_connectivity(tdim, tdim)
topology, cell_types, geometry = plot.vtk_mesh(mesh, tdim)
grid = pyvista.UnstructuredGrid(topology, cell_types, geometry)

plotter = pyvista.Plotter()
plotter.add_mesh(grid, show_edges=True)
plotter.view_xy()
if not pyvista.OFF_SCREEN:
    plotter.show()
else:
    figure = plotter.screenshot("mesh.png")

EmbeddableWidget(value='<iframe srcdoc="<!DOCTYPE html>\n<html>\n  <head>\n    <meta http-equiv=&quot;Content-…

Similarly, the `create_unit_cube` function creates a 3D mesh of the unit cube, subdivided into a specified number of cells along the x, y, and z directions. It is useful for testing and developing 3D simulations.

In [36]:
from dolfinx.mesh import create_unit_cube
from mpi4py import MPI

nx, ny, nz = 10, 10, 10  # Number of divisions in x, y, and z directions
mesh = create_unit_cube(MPI.COMM_WORLD, nx, ny, nz)

In [37]:
tdim = mesh.topology.dim

mesh.topology.create_connectivity(tdim, tdim)
topology, cell_types, geometry = plot.vtk_mesh(mesh, tdim)
grid = pyvista.UnstructuredGrid(topology, cell_types, geometry)

plotter = pyvista.Plotter()
plotter.add_mesh(grid, show_edges=True)
plotter.view_isometric()
if not pyvista.OFF_SCREEN:
    plotter.show()
else:
    figure = plotter.screenshot("mesh.png")

EmbeddableWidget(value='<iframe srcdoc="<!DOCTYPE html>\n<html>\n  <head>\n    <meta http-equiv=&quot;Content-…

The `create_rectangle` function enables the definition of a 2D rectangular mesh over any axis-aligned domain by specifying custom corner coordinates, unlike `create_unit_square`, which has a fixed domain. This provides more flexibility for modelling geometries of arbitrary size.

In [38]:
from dolfinx.mesh import create_rectangle
from mpi4py import MPI

mesh = create_rectangle(MPI.COMM_WORLD, [[0, 0], [2, 1]], [20, 10])

In [39]:
tdim = mesh.topology.dim

mesh.topology.create_connectivity(tdim, tdim)
topology, cell_types, geometry = plot.vtk_mesh(mesh, tdim)
grid = pyvista.UnstructuredGrid(topology, cell_types, geometry)

plotter = pyvista.Plotter()
plotter.add_mesh(grid, show_edges=True)
plotter.view_isometric()
if not pyvista.OFF_SCREEN:
    plotter.show()
else:
    figure = plotter.screenshot("mesh.png")

EmbeddableWidget(value='<iframe srcdoc="<!DOCTYPE html>\n<html>\n  <head>\n    <meta http-equiv=&quot;Content-…

All of the built-in mesh creation functions in DOLFINx, such as `create_unit_square`, `create_unit_cube`, and `create_rectangle`, accept a `cell_type` argument. This specifies the type of elements used to construct the mesh.

Common options include:

- `"triangle"` for 2D triangular elements  
- `"quadrilateral"` for 2D quadrilateral elements  
- `"tetrahedron"` for 3D tetrahedral elements  
- `"hexahedron"` for 3D hexahedral (brick-like) elements

Choosing the appropriate `cell_type` is important, as it determines the structure of the mesh and which finite elements can be used later in the simulation.

In [40]:
from dolfinx.mesh import CellType

mesh = create_unit_square(MPI.COMM_WORLD, 10, 10, cell_type=CellType.quadrilateral)

## Mesh transformations

In some cases, it is useful to apply transformations to a mesh's coordinates—such as scaling, translating, or rotating the domain. This can help with unit conversion, adjusting the position of the mesh, or aligning it with physical geometry.

These operations can be performed directly by modifying the `.geometry.x` attribute of a DOLFINx mesh, which stores the coordinate array of all mesh vertices.

### Scaling

To scale a mesh uniformly or anisotropically, you can multiply the coordinate array by a scalar or a vector. This changes the physical size of the domain without altering its topological structure.

In [42]:
mesh = create_unit_square(MPI.COMM_WORLD, 10, 10)

mesh.geometry.x[:] *= 1e-3

The following example shows how to scale only the x-dimension of a mesh, effectively compressing the domain in one direction:

In [43]:
mesh = create_unit_square(MPI.COMM_WORLD, 10, 10)

mesh.geometry.x[:, 0] *= 0.5

This creates a 2D unit square mesh with 10 divisions in each direction. The second line accesses the mesh’s geometry array and modifies only the first column, which corresponds to the x-coordinates of all vertices. Multiplying these values by 0.5 scales the mesh in the x-direction, transforming the original square into a rectangle. This approach is useful when you need to stretch or compress a mesh in a single direction, for example to simulate an anisotropic domain or convert units along one axis.

In [44]:
tdim = mesh.topology.dim

mesh.topology.create_connectivity(tdim, tdim)
topology, cell_types, geometry = plot.vtk_mesh(mesh, tdim)
grid = pyvista.UnstructuredGrid(topology, cell_types, geometry)

plotter = pyvista.Plotter()
plotter.add_mesh(grid, show_edges=True)
plotter.view_isometric()
if not pyvista.OFF_SCREEN:
    plotter.show()
else:
    figure = plotter.screenshot("mesh.png")

EmbeddableWidget(value='<iframe srcdoc="<!DOCTYPE html>\n<html>\n  <head>\n    <meta http-equiv=&quot;Content-…

### Translating

To move (translate) a mesh to a different position, simply add a constant offset to all coordinates:

In [45]:
mesh = create_unit_square(MPI.COMM_WORLD, 10, 10)

# Move mesh 10 units in x-direction
mesh.geometry.x[:, 0] += 10

### Rotating

Example 1: Rotating a Mesh by 90 Degrees via Coordinate Swapping

This method rotates a 2D mesh 90 degrees counter-clockwise by swapping its x- and y-coordinates.

The mesh is originally defined over the domain $([0, 2] \times [0, 1])$. After swapping coordinates, the domain becomes $([0, 1] \times [0, 2])$, effectively rotating the mesh about the origin.

This approach is simple and efficient but limited to 90° rotations and rotation about the origin.

```{note} Note
The rotation is performed about the origin $(0, 0)$. If your mesh is defined elsewhere, you may need to shift it to the origin before rotation and shift it back afterward.
```

In [46]:
# Create a rectangular mesh over the domain [0, 2] x [0, 1]
mesh = create_rectangle(MPI.COMM_WORLD, [[0, 0], [2, 1]], [20, 10])

# Store a copy of the x-coordinates
x_vals = mesh.geometry.x[:, 0].copy()

# Swap x and y to perform a 90-degree rotation
mesh.geometry.x[:, 0] = mesh.geometry.x[:, 1].copy()
mesh.geometry.x[:, 1] = x_vals

In [47]:
tdim = mesh.topology.dim

mesh.topology.create_connectivity(tdim, tdim)
topology, cell_types, geometry = plot.vtk_mesh(mesh, tdim)
grid = pyvista.UnstructuredGrid(topology, cell_types, geometry)

plotter = pyvista.Plotter()
plotter.add_mesh(grid, show_edges=True)
plotter.view_xy()

# show orientation
plotter.show_axes()

if not pyvista.OFF_SCREEN:
    plotter.show()
else:
    figure = plotter.screenshot("mesh.png")

EmbeddableWidget(value='<iframe srcdoc="<!DOCTYPE html>\n<html>\n  <head>\n    <meta http-equiv=&quot;Content-…

Example 2: Rotating a Mesh by an Arbitrary Angle Using `scipy`

To rotate a mesh by any angle, you can apply a rotation matrix using the `Rotation` class from `scipy.spatial.transform`.

In this example, the mesh is rotated by 30 degrees counter-clockwise about the origin. The rotation is performed by applying the rotation matrix to all mesh coordinates.

If your mesh is not centred at the origin, you should translate it so the rotation point aligns with the origin before rotating, and translate it back afterward.

In [48]:
from scipy.spatial.transform import Rotation

mesh = create_unit_square(MPI.COMM_WORLD, 10, 10)

# Define rotation angle in degrees
degrees = 30

# Create a rotation object for rotation about the z-axis
rotation = Rotation.from_euler("z", degrees, degrees=True)

# Apply rotation to all mesh coordinates
mesh.geometry.x[:, :] = rotation.apply(mesh.geometry.x)

In [49]:
tdim = mesh.topology.dim

mesh.topology.create_connectivity(tdim, tdim)
topology, cell_types, geometry = plot.vtk_mesh(mesh, tdim)
grid = pyvista.UnstructuredGrid(topology, cell_types, geometry)

plotter = pyvista.Plotter()
plotter.add_mesh(grid, show_edges=True)
plotter.view_xy()

# show orientation
plotter.show_axes()

if not pyvista.OFF_SCREEN:
    plotter.show()
else:
    figure = plotter.screenshot("mesh.png")

EmbeddableWidget(value='<iframe srcdoc="<!DOCTYPE html>\n<html>\n  <head>\n    <meta http-equiv=&quot;Content-…

## Passing a dolfinx mesh to FESTIM

To pass a dolfinx mesh to a FESTIM model, simply pass it to a `festim.Mesh` object:

In [50]:
import festim as F

fenics_mesh = create_unit_square(MPI.COMM_WORLD, 10, 10)

festim_mesh = F.Mesh(fenics_mesh)

## Defining subdomains

Now that we know how to pass a dolfinx mesh to FESTIM, we need to mark mesh entities to define subdomains (volume subdomains or surface subdomains).

```{note}
This will be greatly simplified once [PR #1005](https://github.com/festim-dev/FESTIM/pull/1005) is merged.
```

### Surface subdomains

To define surface subdomains, we make use of the ``festim.SurfaceSubdomain`` class and override the ``locate_boundary_facet_indices()`` method:

In [51]:
from dolfinx.mesh import locate_entities_boundary
import numpy as np


class TopSurface(F.SurfaceSubdomain):
    def locate_boundary_facet_indices(self, mesh):
        fdim = mesh.topology.dim - 1

        locator = lambda x: np.isclose(x[1], 1.0)

        indices = locate_entities_boundary(mesh, fdim, locator)
        return indices


class BottomSurface(F.SurfaceSubdomain):
    def locate_boundary_facet_indices(self, mesh):
        fdim = mesh.topology.dim - 1

        locator = lambda x: np.isclose(x[1], 0.0)

        indices = locate_entities_boundary(mesh, fdim, locator)
        return indices


top_surface = TopSurface(id=1)
bottom_surface = BottomSurface(id=2)

### Volume subdomains

Similarily for volume subdomains, we override the ``locate_subdomain_entities()`` method:

In [52]:
from dolfinx.mesh import locate_entities

# volume subdomains need a material
material = F.Material(name="test_material", D_0=1, E_D=0)


class TopVolume(F.VolumeSubdomain):
    def locate_subdomain_entities(self, mesh):
        locator = lambda x: x[1] >= 0.5
        entities = locate_entities(mesh, mesh.topology.dim, locator)
        return entities


class BottomVolume(F.VolumeSubdomain):
    def locate_subdomain_entities(self, mesh):
        locator = lambda x: x[1] <= 0.5
        entities = locate_entities(mesh, mesh.topology.dim, locator)
        return entities


top_volume = TopVolume(id=1, material=material)
bottom_volume = BottomVolume(id=2, material=material)

### Visualise subdomains

Subdomains can be visualised by passing them to a FESTIM problem and then call ``define_meshtags_and_measures()``. This will generate ``dolfinx.mesh.MeshTags`` objects that can then be used for visualisation:

In [53]:
my_model = F.HydrogenTransportProblem()
my_model.mesh = festim_mesh
my_model.subdomains = [top_surface, bottom_surface, top_volume, bottom_volume]

my_model.define_meshtags_and_measures()

print(my_model.facet_meshtags)
print(my_model.volume_meshtags)

``pyvista`` can be used for visualisation:

In [54]:
fdim = my_model.mesh.mesh.topology.dim - 1
tdim = my_model.mesh.mesh.topology.dim
my_model.mesh.mesh.topology.create_connectivity(fdim, tdim)
topology, cell_types, x = plot.vtk_mesh(
    my_model.mesh.mesh, tdim, my_model.volume_meshtags.indices
)

p = pyvista.Plotter(window_size=[800, 800])
grid = pyvista.UnstructuredGrid(topology, cell_types, x)
grid.cell_data["Cell Marker"] = my_model.volume_meshtags.values
grid.set_active_scalars("Cell Marker")
p.add_mesh(grid, show_edges=True)
if pyvista.OFF_SCREEN:
    figure = p.screenshot("volume_marker.png")
p.show()

EmbeddableWidget(value='<iframe srcdoc="<!DOCTYPE html>\n<html>\n  <head>\n    <meta http-equiv=&quot;Content-…

In [55]:
my_model.mesh.mesh.topology.create_connectivity(fdim, tdim)
topology, cell_types, x = plot.vtk_mesh(
    my_model.mesh.mesh, fdim, my_model.facet_meshtags.indices
)

p = pyvista.Plotter(window_size=[800, 800])
grid = pyvista.UnstructuredGrid(topology, cell_types, x)
grid.cell_data["Facet Marker"] = my_model.facet_meshtags.values
grid.set_active_scalars("Facet Marker")
p.add_mesh(grid, show_edges=True)
if pyvista.OFF_SCREEN:
    figure = p.screenshot("facet_marker.png")
p.show()

EmbeddableWidget(value='<iframe srcdoc="<!DOCTYPE html>\n<html>\n  <head>\n    <meta http-equiv=&quot;Content-…

The meshtags can also be stored to a VTX file using ``adios4dolfinx`` to be read later by another script:

In [56]:
import adios4dolfinx

my_model.facet_meshtags.name = "facet_tags"

# write
adios4dolfinx.write_meshtags("facet_tags.bp", my_model.mesh.mesh, my_model.facet_meshtags)

# read
ft = adios4dolfinx.read_meshtags("facet_tags.bp", my_model.mesh.mesh, meshtag_name="facet_tags")

## Complete example

### Implementation

In [57]:
import festim as F
from dolfinx.mesh import locate_entities_boundary
from dolfinx.mesh import locate_entities

import numpy as np

fenics_mesh = create_unit_square(MPI.COMM_WORLD, 10, 10)

festim_mesh = F.Mesh(fenics_mesh)


class TopSurface(F.SurfaceSubdomain):
    def locate_boundary_facet_indices(self, mesh):
        fdim = mesh.topology.dim - 1
        indices = locate_entities_boundary(mesh, fdim, lambda x: np.isclose(x[1], 1.0))
        return indices


class BottomSurface(F.SurfaceSubdomain):
    def locate_boundary_facet_indices(self, mesh):
        fdim = mesh.topology.dim - 1
        indices = locate_entities_boundary(mesh, fdim, lambda x: np.isclose(x[1], 0.0))
        return indices


class TopVolume(F.VolumeSubdomain):
    def locate_subdomain_entities(self, mesh):
        entities = locate_entities(
            mesh,
            mesh.topology.dim,
            lambda x: x[1] >= 0.5,
        )
        return entities


class BottomVolume(F.VolumeSubdomain):
    def locate_subdomain_entities(self, mesh):
        entities = locate_entities(
            mesh,
            mesh.topology.dim,
            lambda x: x[1] <= 0.5,
        )
        return entities


material_top = F.Material(D_0=1, E_D=0)
material_bot = F.Material(D_0=2, E_D=0)


top_volume = TopVolume(id=1, material=material_top)
bottom_volume = BottomVolume(id=2, material=material_bot)

top_surface = TopSurface(id=1)
bottom_surface = BottomSurface(id=2)

my_model = F.HydrogenTransportProblem()
my_model.mesh = festim_mesh
my_model.subdomains = [top_surface, bottom_surface, top_volume, bottom_volume]

H = F.Species("H")
my_model.species = [H]

my_model.temperature = 400

my_model.boundary_conditions = [
    F.FixedConcentrationBC(subdomain=top_surface, value=1.0, species=H),
    F.FixedConcentrationBC(subdomain=bottom_surface, value=0.0, species=H),
]

my_model.settings = F.Settings(atol=1e-10, rtol=1e-10, transient=False)

my_model.initialise()
my_model.run()

### Visualisation

In [58]:
hydrogen_concentration = H.solution

topology, cell_types, geometry = plot.vtk_mesh(hydrogen_concentration.function_space)
u_grid = pyvista.UnstructuredGrid(topology, cell_types, geometry)
u_grid.point_data["c"] = hydrogen_concentration.x.array.real
u_grid.set_active_scalars("c")
u_plotter = pyvista.Plotter()
u_plotter.add_mesh(u_grid, show_edges=True)
u_plotter.view_xy()

if not pyvista.OFF_SCREEN:
    u_plotter.show()
else:
    figure = u_plotter.screenshot("concentration.png")

EmbeddableWidget(value='<iframe srcdoc="<!DOCTYPE html>\n<html>\n  <head>\n    <meta http-equiv=&quot;Content-…